In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# prompt
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = PromptTemplate(
    input_variables=["title", "content"],
    template="""
    당신은 경제 및 산업 분석에 능한 리서처입니다.

    아래는 한 기업에 대한 뉴스 기사입니다.
    이력서를 준비 중인 취업 준비생이 해당 기업을 분석하는 데 도움이 되도록, 핵심 내용을 3~4문장으로 요약해 주세요.

    요약은 다음 기준에 맞게 작성해 주세요:
    - 기업의 최근 이슈, 실적, 전략, 사업 방향 등을 중심으로
    - 과장 없이, 사실 기반으로
    - 투자자가 아닌 취준생의 시선에서 기업을 이해하는 데 도움되도록

    [제목]
    {title}

    [내용]
    {content}
    """)                                     


PromptTemplate(input_variables=['content', 'title'], input_types={}, partial_variables={}, template='\n    당신은 경제 및 산업 분석에 능한 리서처입니다.\n\n    아래는 한 기업에 대한 뉴스 기사입니다.\n    이력서를 준비 중인 취업 준비생이 해당 기업을 분석하는 데 도움이 되도록, 핵심 내용을 3~4문장으로 요약해 주세요.\n\n    요약은 다음 기준에 맞게 작성해 주세요:\n    - 기업의 최근 이슈, 실적, 전략, 사업 방향 등을 중심으로\n    - 과장 없이, 사실 기반으로\n    - 투자자가 아닌 취준생의 시선에서 기업을 이해하는 데 도움되도록\n\n    [제목]\n    {title}\n\n    [내용]\n    {content}\n    ')

In [12]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000146D0C4BB30> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000146D157B860> root_client=<openai.OpenAI object at 0x00000146D15E9010> root_async_client=<openai.AsyncOpenAI object at 0x00000146F790B260> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [13]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [7]:
news_json = {
    "title": "삼성전자, 2분기 영업이익 8조 돌파",
    "content": """삼성전자가 2025년 2분기 실적을 발표하며 영업이익 8조 원을 기록했다.
    반도체 부문의 회복과 갤럭시 스마트폰 시리즈의 글로벌 판매 호조가 주된 요인으로 분석된다.
    전년 동기 대비 120% 증가한 수치로, 시장 기대치를 상회한 실적이다.
    삼성전자는 하반기에도 메모리 수요 회복세가 지속될 것으로 내다봤다."""
}

In [14]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [ ]:
from langchain_core.output_parsers import StrOutputParser


# chain 호출
try:
    result = chain.invoke(news_json)
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
삼성전자가 2025년 2분기 실적을 발표했는데, 반도체 부문의 회복과 갤럭시 스마트폰 시리즈의 글로벌 판매 호조로 영업이익이 8조 원을 돌파했습니다. 이는 전년 동기 대비 120% 증가한 수치로, 시장 기대치를 상회한 실적입니다. 삼성전자는 하반기에도 메모리 수요 회복세가 지속될 것으로 전망하고 있습니다. 이를 통해 삼성전자의 반도체와 스마트폰 사업이 안정적인 성장세를 보이고 있음을 알 수 있습니다.


In [16]:
# chain 호출
try:
    result = chain2.invoke(news_json)
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
삼성전자가 2024년 2분기 실적을 발표했는데요, 반도체 부문의 회복과 갤럭시 스마트폰 시리즈의 글로벌 판매 호조로 영업이익이 8조 원을 돌파했습니다. 이는 전년 동기 대비 120% 증가한 수치로, 시장 기대치를 상회한 실적입니다. 삼성전자는 하반기에도 메모리 수요 회복세가 지속될 것으로 전망하고 있습니다.
